<a href="https://colab.research.google.com/github/filipemcbarros/modelos_classificadores_conciliacao/blob/main/Ajuste_Fino_BERTikal_classificacao_conciliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas utilizadas**

In [ ]:
import pandas as pd
from google.colab import drive
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score,roc_curve,accuracy_score,matthews_corrcoef,f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os

!pip install unidecode
!pip install ftfy
!pip install transformers==4.2.2
!pip install pyreadr
!pip install git+https://github.com/felipemaiapolo/legalnlp
!pip install transformers torch scikit-learn pandas
!pip install simpletransformers

# **Importação da biblioteca *simple transformers* para uso pelo Modelo de Linguagem**

In [ ]:
from simpletransformers.language_modeling import (LanguageModelingModel)

## **Método para carregamento do *dataset***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/base_processos/dataset_ajuste_fino.csv")
dataset

# **Carregamento e Transformação dos Dados**

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset['corpus_bruto'], dataset['conciliado'], random_state = 1234, test_size = 0.3)

doc_train = X_train
doc_valid = X_test

In [ ]:
path_dados = '' #path com os dados pré-processados
path_transformados = '' #path para salvar os dados transformados
path_model = '' #path para salvar o modelo com ajuste fino
path_save_model = os.path.join(path_model,'model')
path_results = path_model + 'Resultados/'

In [ ]:
train_file = os.path.join(path_model,'train.txt')
valid_file = os.path.join(path_model,'valid.txt')

In [ ]:
with open(train_file,'w') as file:
  for doc in doc_train.values:
    file.write(doc+'\n')

In [ ]:
with open(valid_file,'w') as file:
  for doc in doc_valid.values:
    file.write(doc+'\n')

# **Parâmetros do Ajuste Fino do Modelo (*Fine-Tuning*)**

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "block_size": 256,
    "max_seq_length": 256,
    "learning_rate": 2e-5,
    "train_batch_size": 8,
    'eval_batch_size': 8,
    "gradient_accumulation_steps": 8,
    "num_train_epochs": 3,
    "mlm": True,
    "mlm_probability": 0.15,
    "sliding_window": True,
    "stride" : 0.8,
    'evaluate_during_training': False,
    'evaluate_during_training_steps': 140000,
    "output_dir": os.path.join(path_model,'model'),
    "save_model_every_epoch": True,
    'hidden_dropout_prob': 0.1,
    'overwrite_output_dir': True
}

In [ ]:
from legalnlp.get_premodel import *

# Fazendo o download do modelo pre-treinado BERTikal e o seu tokenizador
get_premodel('bert')

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim

print(torch.cuda.is_available())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
model = LanguageModelingModel("bert", '/content/BERTikal/', args=train_args, use_cuda=True, cuda_device=0)

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

model.train_model(train_file, eval_file=valid_file, show_running_loss=True)

In [ ]:
results_eval = model.eval_model(valid_file)

# **Carregamento do modelo e do tokenizador treinado com ajuste fino**

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering
import IPython
from IPython.display import Image
from IPython.display import clear_output

bert_model =  BertModel.from_pretrained('/content/model/').to(device)
bert_tokenizer = BertTokenizer.from_pretrained('/content/model/checkpoint-24615-epoch-3/vocab.txt', do_lower_case=False)

clear_output()

# configuração do BERT
bert_model.config

tokenizer = bert_tokenizer
model = bert_model

In [ ]:
data_text = list(dataset['corpus_bruto'])

In [ ]:
# Definir o DataLoader com seus dados
#train_loader = DataLoader(dataset, batch_size=16, shuffle=True) # batch_size(varia de 8-32)


# Aplicando o bert_tokenizer em nosso dataset com um comprimento máximo de 512 tokens
encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")


#Agora temos nossos encoded_input em um dicionário com 3 chaves
encoded_inputs.keys()

In [ ]:
# Enviando os tensores para para a GPU
input_ids = encoded_inputs['input_ids'].to(device)

In [ ]:
from tqdm import tqdm

# Criando o nosso vetor de features
features = []

# Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor

for i in tqdm(range(len(data_text))):

    with torch.no_grad():

      last_hidden_states = bert_model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()

    features.append(last_hidden_states)

# Criando um numpy array com as features extraidas
features = np.array(features)

In [ ]:
df_features = pd.DataFrame(features)
features_label = pd.concat([df_features, dataset['conciliado']], axis = 1)
features_label.shape

In [ ]:
print(features_label)
!pip install catboost
clear_output()

# **Definindo Conjunto de Treinamento e Testes para os modelos**

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_label.drop(columns = ['conciliado']), features_label['conciliado'], random_state = 1234,test_size = 0.3)

# SVC

In [ ]:
# Treinamento do modelo SVC Gaussian Kernel
from sklearn.svm import SVC

svcGaussian = SVC(kernel='rbf', C=1000.0)
svcGaussian.fit(X_train,y_train)
target_svc_gaussian = svcGaussian.predict(X_test)

# **Avaliação do modelo SVC**

In [ ]:
print('############# Relatório de Classificação - Modelo SVC com ajuste fino BERT #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test, target_svc_gaussian, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test, target_svc_gaussian)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, target_svc_gaussian)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test, target_svc_gaussian)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test, target_svc_gaussian)))

In [ ]:
print('#### Matriz de Confusão - Modelo SVC com ajuste fino BERT ####')
print()
# Calculate confusion matrix
cf_matrix = confusion_matrix(y_test, target_svc_gaussian)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.title('Matriz de Confusão com ajuste fino BERT')
plt.show()

# **Regressão Logística**

In [ ]:
# Treinamento do modelo Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', C=10.0)
lr.fit(X_train,y_train)
target_lr = lr.predict(X_test)

# **Avaliação do modelo Regressão Logística**

In [ ]:
print('############# Relatório de Classificação - Modelo Regressão Logística com ajuste fino BERT #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test, target_lr, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test, target_lr)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, target_lr)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test, target_lr)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test, target_lr)))

In [ ]:
print('#### Matriz de Confusão - Modelo Regressão Logística com ajuste fino BERT ####')
print()
# Calculate confusion matrix
cf_matrix = confusion_matrix(y_test, target_lr)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.title('Matriz de Confusão com ajuste fino Modelo BERT')
plt.show()

# Árvore de Decisão

In [ ]:
# Treinamento do modelo Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train,y_train)
target_dtc = dtc.predict(X_test)

# **Avaliação do modelo Árvore de Decisão**

In [ ]:
print('############# Relatório de Classificação - Modelo Árvore de Decisão com ajuste fino BERT #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test, target_dtc, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test, target_dtc)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, target_dtc)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test, target_dtc)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test, target_dtc)))

In [ ]:
print('#### Matriz de Confusão - Modelo Árvore de Decisão com ajuste fino BERT ####')
print()
# Calculate confusion matrix
cf_matrix = confusion_matrix(y_test, target_dtc)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.title('Matriz de Confusão com ajuste fino BERT')
plt.show()

# XGBoost

In [ ]:
# Treinamento do modelo XGBoost
import xgboost as xgb

# Convert the data into DMatrix format, which is required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'criterion':'squared_error',
    'max_depth': 7,
    'learning_rate': 0.3,
    'n_estimators': 100
}

# Train the Boosted Trees model
bst = xgb.train(params, dtrain)

# Make predictions on the test set
target_xgb = bst.predict(dtest)

# **Avaliação do modelo XGBoost**

In [ ]:
# Definir um limite de decisão (exemplo: 0.5)
limite_decisao = 0.5

# Converter as previsões em classes binárias
previsoes_binarias = [1 if predicao > limite_decisao else 0 for predicao in target_xgb]

# Converter os rótulos reais em classes binárias
rotulos_binarios = [1 if rotulo > limite_decisao else 0 for rotulo in y_test]

print('############# Relatório de Classificação - Modelo XGBoost com ajuste fino BERT #############')
print()

print("Relatório de Classificação:\n", classification_report(rotulos_binarios, previsoes_binarias, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(rotulos_binarios, previsoes_binarias)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(rotulos_binarios, previsoes_binarias)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(rotulos_binarios, previsoes_binarias)))
print("F1-Score:  {:.4f}\n".format(f1_score(rotulos_binarios, previsoes_binarias)))

In [ ]:
print('#### Matriz de Confusão - Modelo XGBoost com ajuste fino BERT ####')
print()
# Calculate confusion matrix
cf_matrix = confusion_matrix(y_test, previsoes_binarias)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.title('Matriz de Confusão com ajuste fino BERT')
plt.show()